In [1]:
#notebook to clean/modify data to a suitable format
import pandas as pd
import numpy as np
import math

In [3]:
#import data
#https://www.kaggle.com/edumucelli/spotifys-worldwide-daily-song-ranking from user 'Eduardo'
df = pd.read_csv('data.csv')

#fix column names
colname_new = []
for col in df.columns:
    colname_new.append(col.lower().replace(' ','_'))
df.columns = colname_new

#extract id from url and rename col
df['url'] = df['url'].str[-22:]
df = df.rename(columns={'url': 'id'})
df

,position,track_name,artist,streams,id,date,region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,3AEZUABDXNtecAOSC1qTfo,2017-01-01,ec
1,2,Chantaje,Shakira,19270,6mICuAdrwEjh6Y6lroV2Kg,2017-01-01,ec
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,3QwBODjSEzelZyVjxPOHdq,2017-01-01,ec
3,4,Vente Pa' Ca,Ricky Martin,14954,7DM4BPaS7uofFul3ywMe46,2017-01-01,ec
4,5,Safari,J Balvin,14269,6rQSrBHf7HlZjtcMZ4S4bO,2017-01-01,ec
...,...,...,...,...,...,...,...
3441192,196,More Than You Know,Axwell /\ Ingrosso,2126,71bBFbfn2OBK5QwUJSLS44,2018-01-09,hk
3441193,197,Issues,Julia Michaels,2117,7vu0JkJh0ldukEYbTVcqd0,2018-01-09,hk
3441194,198,Tuo Fei Lun,Eason Chan,2112,0pmGleExiQyd0HxV5kAOwK,2018-01-09,hk
3441195,199,面具,Alfred Hui,2102,58SUsgamkW6vFFq0YNKGrf,2018-01-09,hk


In [4]:
df[df['region'] == 'global']

,position,track_name,artist,streams,id,date,region
3116643,1,Starboy,The Weeknd,3135625,5aAx2yezTd8zXrkmtKl66Z,2017-01-01,global
3116644,2,Closer,The Chainsmokers,3015525,7BKLCZ1jbUBVqRi2FVlTVw,2017-01-01,global
3116645,3,Let Me Love You,DJ Snake,2545384,4pdPtRcBmOSQDlJ3Fk945m,2017-01-01,global
3116646,4,Rockabye (feat. Sean Paul & Anne-Marie),Clean Bandit,2356604,5knuzwU65gJK7IF5yJsuaW,2017-01-01,global
3116647,5,One Dance,Drake,2259887,1xznGGDReH1oQq0xzbwXa3,2017-01-01,global
...,...,...,...,...,...,...,...
3190438,196,Delicate,Taylor Swift,507893,6NFyWDv5CjfwuzoCkw47Xf,2018-01-09,global
3190439,197,Look At Me!,XXXTENTACION,505171,6eYxmK9fdL4hwLnXG2zY3s,2018-01-09,global
3190440,198,The Scientist,Coldplay,504066,75JFxkI2RXiU7L9VXzMkle,2018-01-09,global
3190441,199,'Till I Collapse,Eminem,501337,6yr8GiTHWvFfi4o6Q5ebdT,2018-01-09,global


In [96]:
#need to convert country codes from ISO 2 to ISO 3 to use chloropleth map
codes = pd.read_csv('country_codes.csv')
codes

,country,code_2,code_3,Unnamed: 3
0,Afghanistan,AF,AFG,4
1,Albania,AL,ALB,8
2,Algeria,DZ,DZA,12
3,American Samoa,AS,ASM,16
4,Andorra,AD,AND,20
...,...,...,...,...
244,Western Sahara,EH,ESH,732
245,Yemen,YE,YEM,887
246,Zambia,ZM,ZMB,894
247,Zimbabwe,ZW,ZWE,716


In [7]:
#convert regions into ISO 3
df.region = df.region.str.upper()
df_global = df[df['region'] == 'GLOBAL']
df = df.merge(codes[['code_2','code_3']],left_on='region',right_on='code_2',how='left').drop(['region','code_2'],axis=1)
#df['code_3'] '
df = df.rename(columns={'code_3' : 'region'})
df

,position,track_name,artist,streams,id,date,region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,3AEZUABDXNtecAOSC1qTfo,2017-01-01,ECU
1,2,Chantaje,Shakira,19270,6mICuAdrwEjh6Y6lroV2Kg,2017-01-01,ECU
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,3QwBODjSEzelZyVjxPOHdq,2017-01-01,ECU
3,4,Vente Pa' Ca,Ricky Martin,14954,7DM4BPaS7uofFul3ywMe46,2017-01-01,ECU
4,5,Safari,J Balvin,14269,6rQSrBHf7HlZjtcMZ4S4bO,2017-01-01,ECU
...,...,...,...,...,...,...,...
3441192,196,More Than You Know,Axwell /\ Ingrosso,2126,71bBFbfn2OBK5QwUJSLS44,2018-01-09,HKG
3441193,197,Issues,Julia Michaels,2117,7vu0JkJh0ldukEYbTVcqd0,2018-01-09,HKG
3441194,198,Tuo Fei Lun,Eason Chan,2112,0pmGleExiQyd0HxV5kAOwK,2018-01-09,HKG
3441195,199,面具,Alfred Hui,2102,58SUsgamkW6vFFq0YNKGrf,2018-01-09,HKG


In [18]:
#remove global/missing songs (separate df for global)
df = df.drop(df[df['region'].isna()].index)
df = df.dropna()

In [ ]:
#add artist nationalities to df
artist_loc = pd.read_csv('artist-locations.csv')
df = df.merge(artist_loc,left_on='artist',right_on='artist').rename(columns={'country' : 'nationality'})

#create df containing a nationality (index) and how popular its artists are in each other region (columns)
artist_weightings_df = pd.DataFrame(index=df['nationality'].unique())
for nat in df['nationality'].unique():
   artist_weightings_df[nat] = df[df['nationality'] == nat][['region','streams']].groupby('region').sum()
artist_weightings_df.to_csv('artist_weightings.csv',index=True)


In [20]:
#save to csvs
df.to_csv('data_clean.csv',index=False)
df_global.to_csv('data_global.csv',index=False)

In [38]:
#3441197 #df
#3367397 #removed nans
#3441197 - 3367397 == 73800 #total removals expected

In [2]:
df = pd.read_csv('data_clean.csv')
df2 = pd.read_csv('spotify-data/data.csv')

In [102]:
#how many top 10 songs are NOT in the original df (df2)
df_top10 = df[df['position'] <= 10]
df_top10[[not x for x in df_top10['id'].isin(df2['id'])]]['id'].value_counts()


5CtI0qwDJkDQGwXD1H1cLb    5076
0ofbQMrRDsUaVKq2mGLEAb    3512
4aWmUDTfIPGksMNLV2rQP2    3338
2rb5MvYT7ZIxbKW5hfcHx8    3305
3eR23VReFzcdmS7TYCrhCe    3107
                          ... 
0RBLY1OUdwvaTgFZhULRnG       1
4QOTOTJsJjgrAf0BgnfaRF       1
5Sg8ZU4sLqsTOEd8PuGmvv       1
2UYG331lOP6amEWplE7eno       1
5uMkdWf1H6JEcpcdnPOSZM       1
Name: id, Length: 1282, dtype: int64

In [97]:
len(df['id'].value_counts()) - len(df_top10[[not x for x in df_top10['id'].isin(df2['id'])]]['id'].value_counts())


2036

In [79]:
df['track_name'].value_counts()
df['id'].value_counts()

#len(set(df['track_name']))

7qiZfU4dY1lWllzX7mPBI3    19001
4aWmUDTfIPGksMNLV2rQP2    15928
6RUKPb4LETWmmr3iAEQktW    15878
5knuzwU65gJK7IF5yJsuaW    15816
3NdDpSvN911VPGivFlV5d0    15082
                          ...  
5UTkoeLEHv0zPy6XtKsCZK        1
2UheWm8tYwjkkeEBqJDMd2        1
2P4OICZRVAQcYAV2JReRfj        1
2Qoz5ApumwSHeSL0GFEfUT        1
3SR45x3OBpDUMNnizoZgOZ        1
Name: id, Length: 21746, dtype: int64

In [123]:
df2[df2['year'] == 2017]

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
7754,0.0447,['YoungBoy Never Broke Again'],0.780,180706,0.784,1,4MofYf0f4ijlVV6elUW5S3,0.000000,1,0.1220,-5.039,1,Untouchable,60,2017-05-31,0.1860,85.023,0.43000,2017
7755,0.1350,['Sky Fall SF'],0.190,209938,0.635,0,03E06FTE8LrhTfdOfDBb2q,0.995000,10,0.5510,-16.847,1,Rain Sounds: Raining,64,2017-05-05,0.0669,91.361,0.00001,2017
7756,0.7520,['chief.'],0.548,77922,0.302,0,6gyYI35SD576xiwstBIX7Y,0.893000,10,0.3140,-12.869,1,sorry,65,2017-12-08,0.1200,127.159,0.54600,2017
7757,0.0884,"['Nicki Minaj', 'Drake', 'Lil Wayne']",0.867,272453,0.558,1,7KcGEssn7BnJdTgildK5y0,0.000004,1,0.5410,-6.975,1,No Frauds,64,2017-03-10,0.2580,139.061,0.43200,2017
7758,0.0142,['amazarashi'],0.594,217627,0.951,0,6wZcefZGc6vrd3zLCQoOYe,0.000001,6,0.0965,-1.339,0,Sora Ni Utaeba,66,2017-12-13,0.0696,102.040,0.64100,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169604,0.4040,['NF'],0.797,239480,0.715,0,5n4ZzdO7KtpxsDpoDTa3D6,0.000000,10,0.0891,-6.054,0,Destiny,63,2017-10-06,0.2330,112.044,0.24300,2017
169605,0.1640,['Greta Van Fleet'],0.362,234600,0.928,0,1BfR5GMOgW0peHkPxrwf11,0.000016,2,0.3150,-4.015,1,Safari Song,64,2017-11-10,0.0503,175.868,0.67800,2017
169606,0.1850,['Ramz'],0.450,202827,0.465,0,2U5cq89GCnsR1yixKkC8d5,0.000000,7,0.0782,-7.867,0,Barking,71,2017-12-01,0.2680,70.231,0.78800,2017
169607,0.6890,"['Rain Sounds', 'Mindfulness Meditation Music ...",0.290,130901,0.997,0,7wOv2XP17ZZHzEWoBmbi9t,0.962000,1,0.9130,-22.985,0,Awakening Rain,65,2017-10-15,0.0492,78.730,0.00447,2017


In [122]:
df2[df2['id'].isin(df['id'])]

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
2381,0.88500,['The Crystals'],0.418,204000,0.831,0,2lT2w8qVNDUAzrdO4yCdxE,0.001880,7,0.3450,-7.462,1,Santa Claus Is Coming to Town,22,1963,0.0870,77.934,0.254,1963
5517,0.36700,['Mariah Carey'],0.362,266933,0.505,0,3B7FO3kJ5kv3mX7yiaB7sT,0.000006,4,0.1540,-7.208,1,O Holy Night,43,1994-11-01,0.0287,174.066,0.192,1994
5747,0.00158,['Robert Miles'],0.621,453400,0.648,0,4d6HyxARNCeUEw3sDntLb5,0.824000,0,0.0532,-11.787,1,Children - Full Length,50,1996-07-11,0.0358,137.070,0.213,1996
5847,0.30500,['Five For Fighting'],0.523,222160,0.464,0,3FvCy00Wz75HmhlXPMmnOx,0.000000,0,0.0992,-8.912,1,Superman (It's Not Easy),36,1997,0.0288,106.547,0.176,1997
6058,0.12200,['Steve Chou'],0.501,344200,0.511,0,4mronxcllhfyhBRqyZi8kU,0.000000,7,0.2790,-9.836,0,黃昏,53,2000-12-25,0.0291,78.045,0.113,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169592,0.00430,['Lil Uzi Vert'],0.623,173798,0.580,1,5tRmoZRoupvMYPV1oezFrH,0.000000,8,0.3940,-7.076,1,X,61,2017-08-25,0.3320,163.159,0.301,2017
169596,0.48400,['Vince Staples'],0.898,198133,0.547,0,6eL1ncVDtQHgSm3E288TS7,0.000010,3,0.3700,-5.707,0,Big Fish,62,2017-06-23,0.3130,100.010,0.763,2017
169597,0.04380,['21 Savage'],0.839,271307,0.437,1,1vc2YF7ZvUtwr45HNpdGch,0.000003,5,0.1310,-11.344,0,Numb,63,2017-09-23,0.2850,82.008,0.464,2017
169603,0.51000,"['Tyler, The Creator', 'Jaden']",0.543,237000,0.572,1,5SxlUF7J8tyFIEF22EomeP,0.000007,11,0.2800,-8.974,1,Pothole (feat. Jaden Smith),63,2017-07-21,0.3380,89.386,0.569,2017


In [23]:
df2['artists'][0]

&quot;[&#39;Carl Woitschach&#39;]&quot;

In [88]:
df2[df2['id'] == '5UEnHoDYpsxlfzWLZIc7LD']

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year


In [107]:
 #test merge of old dataset (incomplete)
 df_top10.merge(df2,on='id').drop_duplicates(subset=['id'])

,position,track_name,artist,streams,id,date,region,acousticness,artists,danceability,...,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,1,Reggaetón Lento (Bailemos),CNCO,19272,3AEZUABDXNtecAOSC1qTfo,2017-01-01,ECU,0.40000,['CNCO'],0.761,...,0.176,-3.073,0,Reggaetón Lento (Bailemos),74,2016-08-26,0.0502,93.974,0.710,2016
1582,2,Chantaje,Shakira,19270,6mICuAdrwEjh6Y6lroV2Kg,2017-01-01,ECU,0.18700,"['Shakira', 'Maluma']",0.852,...,0.159,-2.921,0,Chantaje (feat. Maluma),76,2017-05-26,0.0776,102.034,0.907,2017
3120,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,3QwBODjSEzelZyVjxPOHdq,2017-01-01,ECU,0.05590,"['Zion & Lennox', 'J Balvin']",0.832,...,0.440,-5.429,1,Otra vez (feat. J Balvin),73,2016-09-30,0.1000,96.016,0.704,2016
4920,4,Vente Pa' Ca,Ricky Martin,14954,7DM4BPaS7uofFul3ywMe46,2017-01-01,ECU,0.00431,"['Ricky Martin', 'Maluma']",0.663,...,0.101,-4.070,0,Vente Pa' Ca (feat. Maluma),73,2016-09-22,0.2260,99.935,0.533,2016
5340,8,Closer,The Chainsmokers,9244,7BKLCZ1jbUBVqRi2FVlTVw,2017-01-03,ECU,0.41400,"['The Chainsmokers', 'Halsey']",0.748,...,0.111,-5.599,1,Closer,84,2016-07-29,0.0338,95.010,0.661,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77196,10,No Te Hagas,Jory Boy,5842,5hKjEpYrJIJs6uPl7zw6Y6,2017-03-31,DOM,0.03530,"['Jory Boy', 'Bad Bunny']",0.748,...,0.315,-3.812,0,No Te Hagas,66,2017-03-11,0.0477,131.972,0.433,2017
77223,10,Me Mata,Bad Bunny,6417,2wgDAPgBhFA7pnIbm0YOhP,2017-05-15,DOM,0.55100,"['Bad Bunny', 'Mambo Kingz', 'DJ Luian', 'Arca...",0.841,...,0.119,-11.467,1,Me Mata,68,2017-04-11,0.2050,114.028,0.686,2017
77271,6,Sobredosis,Romeo Santos,9076,1WbXIpiGBJkCiv3hn40iPB,2017-07-21,DOM,0.20700,"['Romeo Santos', 'Ozuna']",0.759,...,0.230,-8.085,0,Sobredosis (feat. Ozuna),71,2017-07-21,0.0511,128.010,0.682,2017
77277,10,Don't Leave,Snakehips,115530,0Zx8khUcEfCFK2AEoIhC92,2017-02-15,AUS,0.22200,"['Snakehips', 'MØ']",0.657,...,0.256,-3.402,1,Don't Leave,67,2017-01-06,0.0753,122.381,0.338,2017


In [10]:
#Create bar plot race dataframe
#import global data with date as index
df_global = pd.read_csv('data_global.csv',index_col=5)
#take top 10 songs per day
df_fig = df_global[df_global['position'] <= 10]
#create a column for each song, with each row being date
df_dates = pd.DataFrame(index=set(df_fig.index))
df_dates = df_dates.sort_index()
for song in set(df_fig['track_name']):
    df_song = df_fig[df_fig['track_name'] == song]
    df_dates[song] = df_song['streams']
#remove null columns
df_dates = df_dates.dropna(axis='columns',how='all')
df_dates = df_dates.fillna(0)
#replace long name
df_dates = df_dates.rename(columns={'I Don’t Wanna Live Forever (Fifty Shades Darker) - From "Fifty Shades Darker (Original Motion Picture Soundtrack)"' : "I Don't Wanna Live Forever"})
df_dates.to_csv('data_dates.csv',index_label='index')

In [215]:
#get sum of streams for each region for each artist
a = pd.DataFrame(df.groupby(['artist','region'])['streams'].sum()).reset_index(level=[1])
a = a.pivot_table(values='streams',index=a.index,columns='region',aggfunc='first')
a.to_csv('artist_streams.csv')

In [221]:
s = pd.DataFrame(df.groupby(['track_name','region'])['streams'].sum()).reset_index(level=[1]).pivot_table(values='streams',index=a.index,columns='region',aggfunc='first')
s.to_csv('song_streams.csv')

In [176]:
#get country of origin of popular artists
df_artist = pd.read_csv('artists.csv')
df_artist = df_artist.drop(columns=['mbid','listeners_lastfm','scrobbles_lastfm','ambiguous_artist'])

In [178]:
#df_artist[df_artist['artist_mb'] != df_artist['artist_lastfm']]
#Get all MB artists in dataframe, and use lastfm artists as backup where missing
df_artist['artist'] = df_artist['artist_mb'].where(df_artist['artist_mb'].isin(df['artist']),df_artist['artist_lastfm'])

#Find duplicate artists
df_artist['artist'].value_counts()[df_artist['artist'].value_counts() > 1]

Angel            50
Alex             37
Aura             34
Oasis            30
Nina             29
                 ..
First Aid Kit     2
Xen               2
Aegis             2
Snowstorm         2
Terri             2
Name: artist, Length: 736, dtype: int64

In [171]:
#test function to aggregate duplicate artists


#aggregate duplicate artists
duplicates = df_artist.groupby(df_artist['artist']).agg({'artist' : ','.join})
#keep only duplicate artists
duplicates = duplicates[duplicates.index != duplicates['artist']]

In [172]:
#Shows all duplicate artists in df_artist
df_artist.where(df_artist['artist'].isin(duplicates.index)).dropna()

,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,artist
7,Nirvana,Nirvana,United States,United States,rock; alternative rock; 90s; punk; american; e...,Grunge; rock; alternative; alternative rock; 9...,Nirvana
8,Muse,Muse,United Kingdom,United Kingdom,rock; electronic; synthpop; alternative rock; ...,alternative rock; rock; alternative; Progressi...,Muse
9,Queen,Queen,United Kingdom,United Kingdom,rock; progressive rock; 70s; 80s; 90s; pop-roc...,classic rock; rock; 80s; hard rock; glam rock;...,Queen
18,Oasis,Oasis,United Kingdom,United Kingdom,rock; alternative rock; british; uk; britanniq...,britpop; rock; british; alternative; indie; se...,Oasis
21,Michael Jackson,Michael Jackson,United States,United States,rock; pop; american; blues; singer; usa; funk;...,pop; 80s; dance; soul; funk; king of pop; mich...,Michael Jackson
...,...,...,...,...,...,...,...
132828,Crush,Crush & Alexandra Ungureanu,Romania,Romania,romanian; romanian duo,Romanian; trance; pop; underrated; electronic;...,Crush
205991,Dick Lee,Dick Lee,Singapore,Singapore,chinese; likedis auto,singapore; pop; singer-songwriter; r&b; Asian;...,Dick Lee
206609,Nicol,Nicol,Latvia,Germany; Latvia,latvian,latvian; pop; rnb; german; female singer; unde...,Nicol
363419,ALMA,Alma Miettinen,Finland,Finland,singer-songwriter,Hip-Hop; female vocalists; rnb; finland; idols...,ALMA


In [71]:
df_artist[df_artist['country_mb'].isna() & df_artist['country_lastfm'].isna()]

,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,artist
3412,Sigala,Sigala,NaN,NaN,dance-pop; tropical house,NaN,Sigala
5665,Calum Scott,Calum Scott,NaN,NaN,NaN,NaN,Calum Scott
5913,[traditional],Traditional,NaN,NaN,special purpose artist; fixme; folk; bogus art...,folk; traditional; ethnic; Bossa Nova; celtic,Traditional
5994,James TW,James Tw,NaN,NaN,NaN,NaN,James TW
6531,Tom Walker,Tom Walker,NaN,NaN,rock; metal; british; guitar; progressive; hea...,indie; metal; electronic; heavy; guitar,Tom Walker
...,...,...,...,...,...,...,...
1418750,iDubbbz,NaN,NaN,NaN,NaN,NaN,iDubbbz
1423663,ÁTOA,NaN,NaN,NaN,NaN,NaN,ÁTOA
1449234,阿達,NaN,NaN,NaN,NaN,NaN,阿達
1459120,Monsta X,NaN,NaN,NaN,NaN,NaN,Monsta X


In [179]:
#drop duplicates and keep only the first artist
df_artist.drop_duplicates(subset=['artist'],keep='first',inplace=True)

In [180]:
#get countries for artists
c = []
for mb,fm in zip(df_artist.country_mb,df_artist.country_lastfm):
    # if pd.isnull(fm):
    #     if not pd.isnull(mb):
    #         c.append(mb.split(';')[0])
    #     else:
    #         c.append(None)
    #mb = str(mb)
    #fm = str(fm)
    if not pd.isnull(fm) and not pd.isnull(mb) and len(mb.split(';')) == 1 and mb in fm:
        c.append(mb)
    elif not pd.isnull(fm) and len(fm.split(';')) == 1:
        c.append(fm)
    elif not pd.isnull(mb) and len(mb.split(';')) == 1:
        c.append(mb)
    elif not pd.isnull(fm):
        c.append(fm.split(';')[0])
    else:
        c.append(None)
#add country list
df_artist['country'] = c
#replace None keyword with NaN and drop artists with no country found
df_artist = df_artist.fillna(value=np.nan).dropna(subset=['country'])[['artist','country']]
df_artist

,artist,country
0,Coldplay,United Kingdom
1,Radiohead,United Kingdom
2,Red Hot Chili Peppers,United States
3,Rihanna,United States
4,Eminem,United States
...,...,...
1465212,WALK THE MOON,United States
1465372,DJ Jazzy Jeff & The Fresh Prince,United States
1465764,Exiled,Germany
1465804,HARUHI,Japan


In [192]:
#to be able to map to country codes
df_artist.replace(['United States','British Virgin Islands','United Kingdom','Netherlands','Philippines','Dominican Republic','Czech Republic','Ivory Coast','Kosovo'],['United States of America (the)','Virgin Islands (British)','United Kingdom of Great Britain and Northern Ireland (the)','Netherlands (the)','Philippines (the)','Dominican Republic (the)','Czechia',"Côte d'Ivoire",'Albania'],inplace=True)
#convert countries to code, then create merge point with df_clean
df_artist = df_artist.merge(codes[['country','code_3']],on='country',how='left').drop('country',axis=1).rename(columns={'code_3' : 'country'})
df_artist.to_csv('artist-locations.csv',index=False)

,artist,country
0,Coldplay,GBR
1,Radiohead,GBR
2,Red Hot Chili Peppers,USA
3,Rihanna,USA
4,Eminem,USA
...,...,...
4316,WALK THE MOON,USA
4317,DJ Jazzy Jeff & The Fresh Prince,USA
4318,Exiled,DEU
4319,HARUHI,JPN
